In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/julian/STT/awni/speech


In [2]:
import json
import random
import torch


import speech.loader as loader
#import speech.models as models


## Config

In [3]:
config_fp = "examples/timit/transducer_config.json"
config2 = "examples/librispeech/config.json"
with open(config_fp, "r") as f:
    config = json.load(f)
with open(config2, "r") as f:
    config2 = json.load(f)

#config["data"]  = config2["data"] #use toy data

#print(config2)
config

{'seed': 2017,
 'save_path': 'examples/timit/models/trans_best',
 'data': {'train_set': 'examples/timit/data/timit/train.json',
  'dev_set': 'examples/timit/data/timit/dev.json',
  'start_and_end': False},
 'optimizer': {'batch_size': 8,
  'epochs': 200,
  'learning_rate': 0.001,
  'momentum': 0.0},
 'model': {'class': 'Transducer',
  'dropout': 0.5,
  'encoder': {'conv': [[8, 5, 32, 2], [8, 5, 32, 1]],
   'rnn': {'dim': 256, 'bidirectional': True, 'layers': 4}},
  'decoder': {'embedding_dim': 256, 'layers': 1}}}

In [24]:
train_set = "/home/ubuntu/Data/LibriSpeech/train-clean-100.json"
dev_set = "/home/ubuntu/Data/LibriSpeech/dev-clean.json"
save_path = "/home/ubuntu/persistent/experiments/rnnt/baseline/first/"
config["save_path"] = save_path
config["data"]["train_set"] = train_set
config["data"]["dev_set"] = dev_set
config

{'seed': 2017,
 'save_path': '/home/ubuntu/persistent/experiments/rnnt/baseline/first/',
 'data': {'train_set': '/home/ubuntu/Data/LibriSpeech/train-clean-100.json',
  'dev_set': '/home/ubuntu/Data/LibriSpeech/dev-clean.json',
  'start_and_end': False},
 'optimizer': {'batch_size': 8,
  'epochs': 200,
  'learning_rate': 0.001,
  'momentum': 0.0},
 'model': {'class': 'Transducer',
  'dropout': 0.5,
  'encoder': {'conv': [[8, 5, 32, 2], [8, 5, 32, 1]],
   'rnn': {'dim': 2048, 'bidirectional': False, 'layers': 4}},
  'decoder': {'embedding_dim': 2048, 'layers': 2}}}

In [25]:
random.seed(config["seed"])
torch.manual_seed(config["seed"])

In [26]:
def nparams(model):
    return sum([p.numel() for p in model.parameters()])


## Model discussion
* The implementation combines the joint and prediction networks into the 'decoder' network 
* SO the ```decoder["layers"]``` is actually the number of layers in the prediction network 



In [27]:
#encoder definition
opt_cfg = config["optimizer"]
data_cfg = config["data"]
model_cfg = config["model"]

enc = model_cfg["encoder"]
dec = model_cfg["decoder"]


HIDDEN_SIZE = 2048
CONV_SIZE = 8 #convolutional channel size
BIDIRECTIONAL = False
DEC_LAYERS = 2

#conv
convs = enc["conv"]
enc["rnn"]["dim"] = HIDDEN_SIZE
enc["rnn"]["bidirectional"] = BIDIRECTIONAL


dec["embedding_dim"] = HIDDEN_SIZE  #NOTE: in strawperson this is 1024. 
                                    #BUT in awni's implementation we add the encoder and 
                                    #Prediction network output
dec["layers"] = 2


out_convs = []
for conv in convs:
    out_c, h, w, s = conv
    out_c = CONV_SIZE
    out_convs.append([out_c, h, w, s])

enc["conv"] = out_convs

for conv in convs:
    out_c, h, w, s = conv
    print("out_c ={}, h={}, w={}, s={}".format(out_c, h, w, s))
print(enc)


with open("configs/strawperson.json", "w") as f:
    json.dump(config, f, indent=2)
config

out_c =8, h=5, w=32, s=2
out_c =8, h=5, w=32, s=1
{'conv': [[8, 5, 32, 2], [8, 5, 32, 1]], 'rnn': {'dim': 2048, 'bidirectional': False, 'layers': 4}}


{'seed': 2017,
 'save_path': '/home/ubuntu/persistent/experiments/rnnt/baseline/first/',
 'data': {'train_set': '/home/ubuntu/Data/LibriSpeech/train-clean-100.json',
  'dev_set': '/home/ubuntu/Data/LibriSpeech/dev-clean.json',
  'start_and_end': False},
 'optimizer': {'batch_size': 8,
  'epochs': 200,
  'learning_rate': 0.001,
  'momentum': 0.0},
 'model': {'class': 'Transducer',
  'dropout': 0.5,
  'encoder': {'conv': [[8, 5, 32, 2], [8, 5, 32, 1]],
   'rnn': {'dim': 2048, 'bidirectional': False, 'layers': 4}},
  'decoder': {'embedding_dim': 2048, 'layers': 2}}}

In [28]:

#START_AND_END = True #Not sure what this does


batch_size = opt_cfg["batch_size"]
preproc = loader.Preprocessor(data_cfg["train_set"], start_and_end=START_AND_END)

model_class = eval("models." + model_cfg["class"])
model1 = model_class(1024, 26,
                        model_cfg)

model2 = model_class(1024,1000,
                        model_cfg)

# train_ldr = loader.make_loader(data_cfg["train_set"],
#                     preproc, batch_size)
# dev_ldr = loader.make_loader(data_cfg["dev_set"],
#                     preproc, batch_size)
print(model1)
print(nparams(model1))
print()

print(model2)
print(model2.conv)
print(nparams(model2))


NameError: name 'START_AND_END' is not defined

In [48]:
sum([p.numel() for p in model2.parameters()])

4651307

In [54]:
38 / 4

9.5